<a href="https://colab.research.google.com/github/baibai25/pytorch-tutorials/blob/main/iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://book.impress.co.jp/books/1119101184

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
iris = datasets.load_iris()

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2
)

In [5]:
# convert tensor
x_train = torch.from_numpy(x_train).type("torch.FloatTensor").to(device)
y_train = torch.from_numpy(y_train).type("torch.LongTensor").to(device)
x_test = torch.from_numpy(x_test).type("torch.FloatTensor").to(device)
y_test = torch.from_numpy(y_test).type("torch.LongTensor").to(device)

In [6]:
# 4 -> 6 -> 3


class MyIris(nn.Module):
    def __init__(self):
        super(MyIris, self).__init__()
        self.l1 = nn.Linear(4, 6)
        self.l2 = nn.Linear(6, 3)

    def forward(self, x):
        h1 = torch.sigmoid(self.l1(x))
        h2 = self.l2(h1)
        return h2

In [7]:
model = MyIris().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

In [8]:
n = 75  # data size
bs = 25  # batch size

model.train()
for i in range(1000):
    idx = np.random.permutation(n)
    for j in range(0, n, bs):
        xtm = x_train[idx[j : (j + bs) if (j + bs) < n else n]]
        ytm = y_train[idx[j : (j + bs) if (j + bs) < n else n]]
        output = model(xtm)
        loss = criterion(output, ytm)
        print(i, j, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

0 0 1.2518341541290283
0 25 1.0454233884811401
0 50 1.1715493202209473
1 0 1.106245756149292
1 25 1.1487716436386108
1 50 1.1160777807235718
2 0 1.1180799007415771
2 25 1.1186633110046387
2 50 1.0771428346633911
3 0 1.0999544858932495
3 25 1.1142656803131104
3 50 1.0746829509735107
4 0 1.0869128704071045
4 25 1.082853078842163
4 50 1.0891495943069458
5 0 1.084328055381775
5 25 1.0939942598342896
5 50 1.0723750591278076
6 0 1.0432114601135254
6 25 1.1181015968322754
6 50 1.074501633644104
7 0 1.1009291410446167
7 25 1.0562304258346558
7 50 1.0685597658157349
8 0 1.0775916576385498
8 25 1.0651870965957642
8 50 1.0438941717147827
9 0 1.0613055229187012
9 25 1.0459425449371338
9 50 1.0576125383377075
10 0 1.0450599193572998
10 25 1.0395880937576294
10 50 1.0649396181106567
11 0 1.0854182243347168
11 25 1.0488582849502563
11 50 1.0210968255996704
12 0 1.0470585823059082
12 25 1.031535267829895
12 50 1.0333623886108398
13 0 1.0088300704956055
13 25 1.0642532110214233
13 50 1.019827127456665


In [9]:
torch.save(model.state_dict(), "myiris.model")
model.load_state_dict(torch.load("myiris.model"))

<All keys matched successfully>

In [10]:
model.eval()
with torch.no_grad():
    output1 = model(x_test)
    ans = torch.argmax(output1, 1)
    print(((y_test == ans).sum().float() / len(ans)).item())

0.9666666984558105
